## Modele de clasificare
Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 6. Cel putin un set de date sa fie cu valori lipsa; pentru un alt set de date care are initial toate valorile, introduceti dvs. in mod artificial valori lipsa, suprascriind un anumit procent din valorile initiale (ex. p=5%, p parametru) cu numpy.nan.

In [ ]:
!pip install numpy
!pip install pandas
!pip install sklearn

In [43]:
import numpy as np
print(f'NumPy version: {np.__version__}')

import pandas as pd
print(f'Pandas version: {pd.__version__}')

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import sklearn as sk
print(f'Sklearn version: {sk.__version__}')

#NumPy version: 1.19.2
#Pandas version: 1.2.3
#Sklearn version: 0.24.1

NumPy version: 1.19.2
Pandas version: 1.2.3
Sklearn version: 0.24.1


In [44]:
le=LabelEncoder()

In [45]:
iris: pd.core.frame.DataFrame = pd.read_csv("Data/iris.data", header=None)
x_iris: np.array = iris.iloc[:, :-1].values
y_iris: np.array = iris[[4]].apply(le.fit_transform).values
y_iris = y_iris.reshape(y_iris.shape[0])

In [46]:
wine:pd.core.frame.DataFrame = pd.read_csv("Data/wine.data", header=None)
x_wine: np.array = wine.iloc[:, 1:].values
y_wine: np.array = wine.iloc[:, 0].values

In [47]:
wdbc: pd.core.frame.DataFrame = pd.read_csv("Data/wdbc.data", header=None)  
x_wdbc: np.array = wdbc.iloc[:, 2:].values
y_wdbc: np.array = wdbc[[1]].apply(le.fit_transform).values
y_wdbc = y_wdbc.reshape(y_wdbc.shape[0])

In [48]:
abalone: pd.core.frame.DataFrame = pd.read_csv("Data/abalone.data", header=None) 
x_abalone: np.array = abalone.iloc[:, 1:].values
y_abalone: np.array = abalone[[0]].apply(le.fit_transform).values
y_abalone = y_abalone.reshape(y_abalone.shape[0])

1. (20 puncte) Aplicati o metoda de missing value imputation, unde este cazul; documentati metoda folosita.
*Resurse*: Pentru missing value imputation, puteti urmari [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html), [How to Handle Missing Data with Python](https://machinelearningmastery.com/handle-missing-data-python/), [fancyimpute](https://github.com/iskandr/fancyimpute), [missingpy](https://github.com/epsilon-machine/missingpy).

*Cerinta*: In cazul in care folositi un pachet ce trebuie instalat (nu face parte din distributia standard anaconda), includeti intr-o celula o comanda de instalare corespunzatoare folosind semn de exclamare, de exemplu:
```python
!pip install missingpy
```
(sursa: [https://github.com/epsilon-machine/missingpy](https://github.com/epsilon-machine/missingpy)). La executia celulei in Jupyter Notebook se instaleaza pachetul, iar in celulele ulterioare importurile din noul pachet functioneaza. 

In [49]:
def random_nans(data_set: np.array, percentage: int = 5) -> np.array:
    """
    Adding NaN values in a dataset.
    
    Args:
        data_set: the numpy.ndarray containing the dataset
        percentage: the percentage(from dataset size) of Nans values to be added in the dataset
        
    Returns:
        the modified dataset
    """
    result_set: np.array = data_set.copy()
    choice: np.array = np.random.choice(result_set.size, int(percentage / 100 * result_set.size), replace=False)
    rows, cols = np.unravel_index(choice, result_set.shape)   
    result_set[rows, cols] = np.nan
    return result_set

In [50]:
x_iris = random_nans(x_iris)
x_wine = random_nans(x_wine)
x_wdbc = random_nans(x_wdbc)
x_abalone = random_nans(x_abalone)

In [51]:
imp = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
x_iris = imp.fit_transform(x_iris)
x_wine = imp.fit_transform(x_wine)
x_wdbc = imp.fit_transform(x_wdbc)
x_abalone = imp.fit_transform(x_abalone)

# 2. (numar de modele * numar de seturi de date \* 1 punct = 20 de puncte) Pentru fiecare set de date aplicati 5 modele de clasificare din scikit learn. Pentru fiecare raportati: acuratete, precision, recall, scorul F1 - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics), [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) - folosind 5 fold cross validation. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare. Rularile se vor face cu valori fixate ale hiperparametrilor. 

In [52]:
def show_metrics_classifier(clf: sk.neighbors._classification.KNeighborsClassifier, x: np.array, y: np.array) -> None:
    """
    Shows the metrics(accuracy, precision, f1, recall) of a classifier.
    
    Args:
        clf: a classifier
        x: np.array containning the dataset information
        y: np.array containning the classification of the data
        
    Returns:
        None
    """
    scoring = ['recall_macro', 'precision_macro', 'accuracy', 'f1_macro']
    scores = cross_validate(clf, x, y, cv=5, scoring=scoring, return_train_score=True)
    df = pd.DataFrame(data={'train_accuracy': scores['train_accuracy'],
                            'train_recall': scores['train_recall_macro'],
                            'train_precision': scores['train_precision_macro'],
                            'train_f1': scores['train_f1_macro'],
                            'test_accuracy': scores['test_accuracy'],
                            'test_recall': scores['test_recall_macro'],
                            'test_precision': scores['test_precision_macro'],
                            'test_f1': scores['test_f1_macro']
                           })
    df.loc['mean'] = df.mean()
    with pd.option_context('expand_frame_repr', False):
        print (df)

In [53]:
print("Metrics for KNeighborsClassifier:")
print("\nIris_Dataset:")
show_metrics_classifier(KNeighborsClassifier(n_neighbors=5),x_iris,y_iris);
print("\nWine_Dataset:")
show_metrics_classifier(KNeighborsClassifier(n_neighbors=5),x_wine ,y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier(KNeighborsClassifier(n_neighbors=5),x_wdbc ,y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier(KNeighborsClassifier(n_neighbors=5),x_abalone ,y_abalone);

Metrics for KNeighborsClassifier:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.950000      0.950000         0.951058  0.949294       0.900000     0.900000        0.902357  0.899749
1           0.933333      0.933333         0.933862  0.932392       1.000000     1.000000        1.000000  1.000000
2           0.950000      0.950000         0.949916  0.949562       0.866667     0.866667        0.866667  0.866667
3           0.933333      0.933333         0.933250  0.932896       0.866667     0.866667        0.875000  0.865320
4           0.950000      0.950000         0.950000  0.950000       0.933333     0.933333        0.944444  0.932660
mean        0.943333      0.943333         0.943617  0.942829       0.913333     0.913333        0.917694  0.912879

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0       

In [54]:
print("Metrics for LogisticRegression:")
print("\nIris_Dataset:")
show_metrics_classifier(LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000), x_iris,y_iris);
print("\nWine_Dataset:")
show_metrics_classifier(LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000), x_wine, y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier(LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000), x_wdbc, y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier(LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000) ,x_abalone, y_abalone);

Metrics for LogisticRegression:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.941667      0.941667         0.941392  0.941237       0.833333     0.833333        0.828956  0.829741
1           0.883333      0.883333         0.881495  0.882041       0.933333     0.933333        0.944444  0.932660
2           0.891667      0.891667         0.892817  0.891044       0.933333     0.933333        0.944444  0.932660
3           0.941667      0.941667         0.941806  0.941437       0.866667     0.866667        0.875000  0.865320
4           0.900000      0.900000         0.900585  0.899937       0.900000     0.900000        0.907407  0.899522
mean        0.911667      0.911667         0.911619  0.911139       0.893333     0.893333        0.900051  0.891980

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0         

In [55]:
print("Metrics for GaussianNB:")
print("\nIris_Dataset:")
show_metrics_classifier(GaussianNB(), x_iris, y_iris);
print("\nWine_Dataset:")
show_metrics_classifier(GaussianNB(), x_wine, y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier(GaussianNB(), x_wdbc, y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier(GaussianNB(), x_abalone, y_abalone);

Metrics for GaussianNB:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.941667      0.941667         0.941229  0.941349       0.800000     0.800000        0.805556  0.797980
1           0.900000      0.900000         0.900569  0.899076       0.966667     0.966667        0.969697  0.966583
2           0.900000      0.900000         0.899230  0.899335       0.900000     0.900000        0.923077  0.897698
3           0.908333      0.908333         0.907468  0.907805       0.866667     0.866667        0.875000  0.865320
4           0.916667      0.916667         0.916667  0.916667       0.933333     0.933333        0.939394  0.931217
mean        0.913333      0.913333         0.913033  0.912846       0.893333     0.893333        0.902545  0.891760

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.9718

In [56]:
print("Metrics for RandomForestClassifier:")
print("\n Iris_Dataset:")
show_metrics_classifier(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), x_iris, y_iris);
print("\n Wine_Dataset:")
show_metrics_classifier(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), x_wine, y_wine);
print("\n Wdbc_Dataset:")
show_metrics_classifier(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), x_wdbc, y_wdbc);
print("\n Abalone_Dataset:")
show_metrics_classifier(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), x_abalone, y_abalone);

Metrics for RandomForestClassifier:

 Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.983333      0.983333         0.984127  0.983323       0.900000     0.900000        0.902357  0.899749
1           0.966667      0.966667         0.966667  0.966667       0.966667     0.966667        0.969697  0.966583
2           0.991667      0.991667         0.991870  0.991665       0.900000     0.900000        0.902357  0.899749
3           0.975000      0.975000         0.975997  0.974768       0.966667     0.966667        0.969697  0.966583
4           0.991667      0.991667         0.991870  0.991665       0.966667     0.966667        0.969697  0.966583
mean        0.981667      0.981667         0.982106  0.981618       0.940000     0.940000        0.942761  0.939850

 Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0   

In [57]:
print("Metrics for DecisionTreeClassifier:")
print("\nIris_Dataset:")
show_metrics_classifier(DecisionTreeClassifier(max_depth=5), x_iris, y_iris);
print("\nWine_Dataset:")
show_metrics_classifier(DecisionTreeClassifier(max_depth=5), x_wine, y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier(DecisionTreeClassifier(max_depth=5), x_wdbc, y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier(DecisionTreeClassifier(max_depth=5), x_abalone, y_abalone);

Metrics for DecisionTreeClassifier:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           1.000000      1.000000         1.000000  1.000000       0.933333     0.933333        0.944444  0.932660
1           1.000000      1.000000         1.000000  1.000000       0.866667     0.866667        0.904762  0.868077
2           1.000000      1.000000         1.000000  1.000000       0.900000     0.900000        0.902357  0.899749
3           1.000000      1.000000         1.000000  1.000000       0.866667     0.866667        0.866667  0.866667
4           0.991667      0.991667         0.991870  0.991665       0.966667     0.966667        0.969697  0.966583
mean        0.998333      0.998333         0.998374  0.998333       0.906667     0.906667        0.917585  0.906747

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0     

3. (numar de modele * numar de seturi de date * 1 punct = 20 de puncte) Raportati performanta fiecarui model, folosind 5 fold cross validation. Pentru fiecare din cele 5 rulari, cautati hiperparametrii optimi folosind 4-fold cross validation. Performanta modelului va fi raportata ca medie a celor  5 rulari. 
    *Observatie:* la fiecare din cele 5 rulari, hiperparametrii optimi pot diferi, din cauza datelor utilizate pentru antrenare/validare. 

In [62]:
def show_metrics_classifier_with_performance(clf, parameter_grid: dict, x: np.array, y:np.array) -> None:
    """
    Shows the metrics(accuracy,precision,f1,recall) of a classifier,by choosing the appropiate hiperparameters.
    
    Args:
        clf: a classifier
        parameter_grid: a dictionary containning the hiperparameters
        x: np.array containning the dataset information
        y: np.array containning the classification of the data
        
    Returns:
        None
    """
    show_metrics_classifier(GridSearchCV(estimator = clf, 
        param_grid=parameter_grid, cv=4, return_train_score=True), x, y)

In [63]:
print("Metrics for KNeighborsClassifier with performance:")
parameter_grid_KNeighborsClassifier = {'n_neighbors': list(range(1, 10)), 'p': [1, 2, 3, 10]}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(KNeighborsClassifier(), parameter_grid_KNeighborsClassifier, x_iris, y_iris);
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(KNeighborsClassifier(), parameter_grid_KNeighborsClassifier, x_wine, y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(KNeighborsClassifier(), parameter_grid_KNeighborsClassifier, x_wdbc, y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(KNeighborsClassifier(), parameter_grid_KNeighborsClassifier, x_abalone, y_abalone);

Metrics for KNeighborsClassifier with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.950000      0.950000         0.951058  0.949294       0.900000     0.900000        0.902357  0.899749
1           1.000000      1.000000         1.000000  1.000000       0.966667     0.966667        0.969697  0.966583
2           0.950000      0.950000         0.949916  0.949562       0.966667     0.966667        0.969697  0.966583
3           0.933333      0.933333         0.936410  0.933106       0.866667     0.866667        0.881410  0.864904
4           0.950000      0.950000         0.950710  0.949969       0.900000     0.900000        0.907407  0.899522
mean        0.956667      0.956667         0.957619  0.956386       0.920000     0.920000        0.926114  0.919468

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision  

In [64]:
print("Metrics for LogisticRegression with performance:")
parameter_grid_LogisticRegression = {'solver' : ['newton-cg', 'sag', 'saga'],
                                     'penalty' : ['l2'],
                                     'C' : [100, 10, 1.0, 0.1, 0.01],
                                     'max_iter' : [10000]}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(LogisticRegression(), parameter_grid_LogisticRegression, x_iris, y_iris);
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(LogisticRegression(), parameter_grid_LogisticRegression, x_wine, y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(LogisticRegression(), parameter_grid_LogisticRegression, x_wdbc, y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(LogisticRegression(), parameter_grid_LogisticRegression, x_abalone, y_abalone);

Metrics for LogisticRegression with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.958333      0.958333         0.958516  0.958327       0.933333     0.933333        0.944444  0.932660
1           0.925000      0.925000         0.925162  0.924988       0.966667     0.966667        0.969697  0.966583
2           0.941667      0.941667         0.943222  0.941585       0.900000     0.900000        0.923077  0.897698
3           0.933333      0.933333         0.933333  0.933333       0.833333     0.833333        0.849817  0.829497
4           0.916667      0.916667         0.916667  0.916667       0.900000     0.900000        0.907407  0.899522
mean        0.935000      0.935000         0.935380  0.934980       0.906667     0.906667        0.918889  0.905192

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   t

In [65]:
print("Metrics for GaussianNB with performance:")
parameter_grid_GaussianNB = {'var_smoothing' : np.logspace(0, -9, num=100)}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(GaussianNB(), parameter_grid_GaussianNB, x_iris, y_iris);
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(GaussianNB(), parameter_grid_GaussianNB, x_wine, y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(GaussianNB(), parameter_grid_GaussianNB, x_wdbc, y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(GaussianNB(), parameter_grid_GaussianNB, x_abalone, y_abalone);

Metrics for GaussianNB with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.941667      0.941667         0.941229  0.941349       0.800000     0.800000        0.805556  0.797980
1           0.883333      0.883333         0.883598  0.882328       0.933333     0.933333        0.944444  0.932660
2           0.900000      0.900000         0.898601  0.898921       0.900000     0.900000        0.923077  0.897698
3           0.908333      0.908333         0.907468  0.907805       0.866667     0.866667        0.875000  0.865320
4           0.908333      0.908333         0.908484  0.908319       0.933333     0.933333        0.939394  0.931217
mean        0.908333      0.908333         0.907876  0.907744       0.886667     0.886667        0.897494  0.884975

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0

In [69]:
print("Metrics for RandomForestClassifier with performance:")
parameter_grid_RandomForestClassifier = {'n_estimators' : [10, 100, 1000], 'max_features' : ['sqrt', 'log2']}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(RandomForestClassifier(), parameter_grid_RandomForestClassifier, x_iris, y_iris);
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(RandomForestClassifier(), parameter_grid_RandomForestClassifier, x_wine, y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(RandomForestClassifier(), parameter_grid_RandomForestClassifier, x_wdbc, y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(RandomForestClassifier(), parameter_grid_RandomForestClassifier, x_abalone, y_abalone);

Metrics for RandomForestClassifier with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           1.000000      1.000000         1.000000  1.000000       0.933333     0.933333        0.944444  0.932660
1           1.000000      1.000000         1.000000  1.000000       0.966667     0.966667        0.969697  0.966583
2           0.991667      0.991667         0.991870  0.991665       0.933333     0.933333        0.944444  0.932660
3           1.000000      1.000000         1.000000  1.000000       0.900000     0.900000        0.911111  0.899327
4           1.000000      1.000000         1.000000  1.000000       0.966667     0.966667        0.969697  0.966583
mean        0.998333      0.998333         0.998374  0.998333       0.940000     0.940000        0.947879  0.939563

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision

In [68]:
print("Metrics for DecisionTreeClassifier with performance:")
parameter_grid_KNeighborsClassifier = {'criterion' : ['gini', 'entropy'], 'max_depth' : [2, 4, 6, 8, 10, 12]}
print("\nIris_Dataset:")
show_metrics_classifier_with_performance(DecisionTreeClassifier(), parameter_grid_KNeighborsClassifier, x_iris, y_iris);
print("\nWine_Dataset:")
show_metrics_classifier_with_performance(DecisionTreeClassifier(), parameter_grid_KNeighborsClassifier, x_wine, y_wine);
print("\nWdbc_Dataset:")
show_metrics_classifier_with_performance(DecisionTreeClassifier(), parameter_grid_KNeighborsClassifier, x_wdbc, y_wdbc);
print("\nAbalone_Dataset:")
show_metrics_classifier_with_performance(DecisionTreeClassifier(), parameter_grid_KNeighborsClassifier, x_abalone, y_abalone);

Metrics for DecisionTreeClassifier with performance:

Iris_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision   test_f1
0           0.950000      0.950000         0.951115  0.949850       0.933333     0.933333        0.944444  0.932660
1           0.991667      0.991667         0.991870  0.991665       0.833333     0.833333        0.888889  0.837473
2           0.950000      0.950000         0.953991  0.949853       0.900000     0.900000        0.902357  0.899749
3           1.000000      1.000000         1.000000  1.000000       0.833333     0.833333        0.835017  0.832916
4           1.000000      1.000000         1.000000  1.000000       0.966667     0.966667        0.969697  0.966583
mean        0.978333      0.978333         0.979395  0.978274       0.893333     0.893333        0.908081  0.893876

Wine_Dataset:
      train_accuracy  train_recall  train_precision  train_f1  test_accuracy  test_recall  test_precision

4. (numar modele * 4 puncte = 20 puncte) Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Daca acelasi algoritm e folosit pentru mai multe seturi de date, puteti face o sectiune separata cu documentarea algoritmilor + trimitere la algoritm. 

#### 1. KNeighborsClassifier
Clasificare K-nearest-neighbor (kNN) este una dintre cele mai fundamentale si simple metode de clasificare fiind printre primele alegeri pentru a invata o clasificare atunci cand nu se stie informatii despre distribuirea datelor. Aceasta metoda a fost dezvoltata din nevoia de a efectua o analiza discriminatoare cand probabilitatea parametrilor de densitate este necunoscuta sau dificil de determinat.Este folosit pentru clasificare si regresie, in ambele cazuri, inputul consista in k cele mai apropiate exemple de antrenare din dataset. Intr-o clasificare outputul este o clasa membru.

In aceata metoda se folosesc k vecini, cei mai apropiati de inregistrarea ce se vrea a ﬁ clasiﬁcata.
Metoda are nevoie de: setul de inregistrari cu clase cunoscute, o metrica (distanta, functie de similaritate) care calculeaza distanta intre doua inregistrari, pe baza valorilor atributelor valoarea k, numarul de vecini cei mai apropiati care sunt considerati.

Pentru clasiﬁcarea unei inregistrari: se calculeaza distanta catre alte inregistrari din setul de antrenare, se identiﬁca cei mai apropiati k vecini, se folosesc etichetele de clasa ale acestor k vecini pentru a estima clasa asociata inregistrarii de test (de exemplu prin considerarea votului majoritar)

class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None, **kwargs)

[KNeighborsClassifier documentatie](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

#### 2. LogisticRegression
Regresia logistica este o clasificare fundamentala. Apartine grupului liniar de clasificatori si este oarecum similar cu regresia polinomiala si liniara. Regresia logistica este rapida si relativ usoara de inteles, fiind usor de interpretat rezultatele. Regresia logistica este folosita pentru estimare de probabilitate conditionatasi clasificare. Initial dezvoltata pentru lucrul cu doua clase, a fost extinsa pentru a discrimina intre oricate clase — regresia logistica multinomiala.

Ca mod de instruire se foloseste invatarea supervizata. Intrarile sunt vectori numerici, iar clasele sunt fie doua (pentru regresia logistica binara), fie mai multe (pentru regresia logistica multinomiala).

class sklearn.linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

[LogisticRegression documentatie](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression)

#### 3. GaussianNB
Algoritmul gaussion Naive Bayes este un algoritm NB special. Este utilizat atunci cand trasaturile au valori continue. De asemenea trebuie asumat faptul ca toate trasaturile urmeaza o distributie gausiana(normal distribution). Acest algoritm este bazat pe probabilitate conditionata. Probabilitatea conditionata ne ajuta sa calculam probabilitatea ca ceva se va intampla, avand in vedere ca altceva deja s-a intamplat.

class sklearn.naive_bayes.GaussianNB(*, priors=None, var_smoothing=1e-09)

[GaussianNB documentatie](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)

####  4. Random forest
Padurile aleatoare sunt o metoda de invatare pentru clasificare, regresie si alte sarcini care opereaza prin construirea unei multitudini de arbori de decizie pe mai multe subseturi de date si care foloseste o medie pentru a imbunatati precizia predictiva.

class sklearn.ensemble.RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)

[RandomForestClassifier_documentatie](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)

####  5. DecisionTreeClassifier
Arborii de decizie (DT) sunt o metoda de invatare neparametrica, supravegheata folosita pentru clasificare si regresie. Scopul lor este de a crea un model care prezice valoarea unei variabile tinta prin invatarea unor reguli simple de decizie deduse din caracteristicile datelor de antrenare. Astfel, fiecare nod intern al arborelui corespunde unui atribut, iar fiecare nod frunza corespunde unei etichete de clasa. Algoritmul plaseaza cel mai bun atribut al setului de date la radacina arborelui. Imparte setul de antrenare in subseturi. Subseturile sunt facute astfel incat fiecare subset sa contina date cu aceeasi valoare pentru un atribut. Se repeta pasii de mai sus pe fiecare subset pana cand se gasesc nodurile frunza pentru toate ramurile arborelui.

class sklearn.tree.DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

[DecisionTreeClassifier_documentaie](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)